In [1]:
import pandas as pd
import datetime as dt

In [2]:
def load_texas_ship():
    box_link = "https://www.dropbox.com/s/95ujsl5fa1r4ko2/prescription_data.zip?dl=1"
    # reference states for shipment analysis
    texas_ref = ["IL", "NY", "OR", "TX"]
    cols_needed = [
        "BUYER_STATE",
        "BUYER_COUNTY",
        "TRANSACTION_DATE",
        "CALC_BASE_WT_IN_GM",
        "MME_Conversion_Factor",
        "MME",
    ]

    data = []
    # read in full shipment data
    ship_iter = pd.read_csv(
        box_link, compression="zip", iterator=True, chunksize=1000000, usecols=cols_needed
    )

    # process chunks for required ref states
    for ship_chunk in ship_iter:
        chunk = ship_chunk[ship_chunk["BUYER_STATE"].isin(texas_ref)]
        data.append(chunk)
    texas_ship_df = pd.concat(data)

    # output to csv for cleaning
    texas_ship_df.to_csv(
        "../20_intermediate_files/texas_shipment_data.csv", encoding="utf-8", index=False
    )

In [3]:
load_texas_ship()
# move texas_shipment_data.csv to dropbox then delete from 20_intermediate_files folder
# add link to dropbox below as tx_box
# run cell below 

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_34228/2324720397.py:21: DtypeWarning: Columns (30,31,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for ship_chunk in ship_iter:
/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_34228/2324720397.py:21: DtypeWarning: Columns (30,31,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for ship_chunk in ship_iter:
/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_34228/2324720397.py:21: DtypeWarning: Columns (30,31,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for ship_chunk in ship_iter:
/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_34228/2324720397.py:21: DtypeWarning: Columns (30,31,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for ship_chunk in ship_iter:
/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_34228/2324720397.py:21: DtypeWarning: Columns

In [5]:
# dropbox links - Sukhpreet add
tx_box = "https://www.dropbox.com/s/fgtnexh3pi8wgd4/texas_shipment_data.csv?dl=1"

tx_df = pd.read_csv(tx_box)

def clean_shipment_month(df, state_name):
    # convert TRANSACTION_DATE into datetime
    df["DATE"] = pd.to_datetime(df["TRANSACTION_DATE"], format="%m%d%Y")
    # find min and max date given by Texas to subset control states
    texas_only = df[df["BUYER_STATE"] == 'TX']
    df_sub = df[(df['DATE'] >= texas_only["DATE"].min()) & (df["DATE"] <= texas_only["DATE"].max())]
    # extract year
    df_sub["MONTH"] = df_sub["DATE"].dt.month

    # make sure there are no missing opioid amounts
    assert df_sub["CALC_BASE_WT_IN_GM"].isna().sum() == 0
    assert df_sub["MME"].isna().sum() == 0

    # could not find an accurate way to impute missing counties, chose to drop
    clean_df = df_sub.dropna(subset=["BUYER_STATE", "BUYER_COUNTY"])

    # group to find total opioids shipped per state-county-year
    grouped = clean_df.groupby(["BUYER_STATE", "BUYER_COUNTY", "MONTH"], as_index=False)[
        "MME"
    ].sum()

    grouped.to_csv(
        f"../20_intermediate_files/{state_name}_shipment_cleaned.csv",
        encoding="utf-8",
        index=False,
    )
    return None


if __name__ == "__main__":
    clean_shipment_month(tx_df, "texas")
